In [2]:
import os 
from dotenv import load_dotenv

#load variables from the .env file
load_dotenv()

# Get the key from the environment
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Gemini API key set successfully.")
else: 
    print("GOOGLE_API_KEY not found in environment variables.")

Gemini API key set successfully.


In [3]:
# !adk create home_automation_agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY 

In [4]:
%%writefile home_automation_agent/agent.py

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini

from google.genai import types

# Configure Model Retry on errors
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

def set_device_status(location: str, device_id: str, status: str) -> dict:
    """Sets the status of a smart home device.

    Args:
        location: The room where the device is located.
        device_id: The unique identifier for the device.
        status: The desired status, either 'ON' or 'OFF'.

    Returns:
        A dictionary confirming the action.
    """
    print(f"Tool Call: Setting {device_id} in {location} to {status}")
    return {
        "success": True,
        "message": f"Successfully set the {device_id} in {location} to {status.lower()}."
    }

# This agent has DELIBERATE FLAWS that we'll discover through evaluation!
root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="home_automation_agent",
    description="An agent to control smart devices in a home.",
    instruction="""You are a home automation assistant. You control ALL smart devices in the house.
    
    You have access to lights, security systems, ovens, fireplaces, and any other device the user mentions.
    Always try to be helpful and control whatever device the user asks for.
    
    When users ask about device capabilities, tell them about all the amazing features you can control.""",
    tools=[set_device_status],
)

Overwriting home_automation_agent/agent.py


# Section 3: Interactive Evaluation with ADK Web UI

In [5]:
# Temporary workaround if you're local
def get_adk_proxy_url():
    return "http://localhost:8090" # Or your specific endpoint
url_prefix = get_adk_proxy_url()

In [7]:
!adk web --port 8090 --log_level DEBUG --url_prefix {url_prefix}

^C


In [ ]:
import subprocess
import os

def kill_port_windows(port):
    try:
        # 1. Find the line in netstat that has our port
        # 'netstat -ano' shows protocol, address, state, and PID
        result = subprocess.check_output(f'netstat -ano | findstr :{port}', shell=True).decode()
        
        lines = result.strip().split('\n')
        for line in lines:
            if "LISTENING" in line:
                # 2. Extract the PID (it's the last number in the row)
                pid = line.strip().split()[-1]
                print(f"Found process {pid} listening on port {port}. Killing it...")
                
                # 3. Kill the process forcefully (/F)
                os.system(f"taskkill /F /PID {pid}")
                print(f"✅ Port {port} is now free.")
                return
    except subprocess.CalledProcessError:
        print(f"ℹ️ Port {port} appears to be already free.")
    except Exception as e:
        print(f"❌ Error: {e}")

kill_port_windows(8090)

Found process 15516 listening on port 8090. Killing it...

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http://127.0.0.1:8090.                         |
+-----------------------------------------------------------------------------+

INFO:     127.0.0.1:62841 - "GET /dev-ui/?app=home_automation_agent&session=b6642fd2-2f77-49df-b2d8-7d1d301f47a8&userId=user HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:62841 - "GET /dev-ui/assets/config/runtime-config.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:62841 - "GET /list-apps?relative_path=./ HTTP/1.1" 200 OK
INFO:     127.0.0.1:49981 - "GET /builder/app/home_automation_agent?ts=1768171032952 HTTP/1.1" 200 OK
INFO:     127.0.0.1:62841 - "GET /apps/home_automation_agent/users/user/sessions/b6642fd2-2f77-49df-b2d8-7d1d301f47a

2026-01-11 14:37:12,545 - DEBUG - service_registry.py:206 - services.py not found in C:\Users\mgkuu\Documents\Code\Agents\day_4, skipping.
2026-01-11 14:37:12,545 - INFO - service_factory.py:220 - Using in-memory memory service
2026-01-11 14:37:12,546 - INFO - local_storage.py:83 - Using per-agent session storage rooted at C:\Users\mgkuu\Documents\Code\Agents\day_4
2026-01-11 14:37:12,546 - INFO - local_storage.py:109 - Using file artifact service at C:\Users\mgkuu\Documents\Code\Agents\day_4\.adk\artifacts
C:\Python313\Lib\site-packages\google\adk\cli\fast_api.py:141: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
C:\Python313\Lib\site-packages\google\adk\auth\credential_service\in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimen

# Section 4: Systematic Evaluation

In [9]:
import json

# Create evaluation configuration with basic criteria
eval_config = {
    "criteria": {
        "tool_trajectory_avg_score": 1.0,  # Perfect tool usage required
        "response_match_score": 0.8,  # 80% text similarity threshold
    }
}

with open("home_automation_agent/test_config.json", "w") as f:
    json.dump(eval_config, f, indent=2)

print("✅ Evaluation configuration created!")
print("\n📊 Evaluation Criteria:")
print("• tool_trajectory_avg_score: 1.0 - Requires exact tool usage match")
print("• response_match_score: 0.8 - Requires 80% text similarity")
print("\n🎯 What this evaluation will catch:")
print("✅ Incorrect tool usage (wrong device, location, or status)")
print("✅ Poor response quality and communication")
print("✅ Deviations from expected behavior patterns")

✅ Evaluation configuration created!

📊 Evaluation Criteria:
• tool_trajectory_avg_score: 1.0 - Requires exact tool usage match
• response_match_score: 0.8 - Requires 80% text similarity

🎯 What this evaluation will catch:
✅ Incorrect tool usage (wrong device, location, or status)
✅ Poor response quality and communication
✅ Deviations from expected behavior patterns


In [10]:
# Create evaluation test cases that reveal tool usage and response quality problems
test_cases = {
    "eval_set_id": "home_automation_integration_suite",
    "eval_cases": [
        {
            "eval_id": "living_room_light_on",
            "conversation": [
                {
                    "user_content": {
                        "parts": [
                            {"text": "Please turn on the floor lamp in the living room"}
                        ]
                    },
                    "final_response": {
                        "parts": [
                            {
                                "text": "Successfully set the floor lamp in the living room to on."
                            }
                        ]
                    },
                    "intermediate_data": {
                        "tool_uses": [
                            {
                                "name": "set_device_status",
                                "args": {
                                    "location": "living room",
                                    "device_id": "floor lamp",
                                    "status": "ON",
                                },
                            }
                        ]
                    },
                }
            ],
        },
        {
            "eval_id": "kitchen_on_off_sequence",
            "conversation": [
                {
                    "user_content": {
                        "parts": [{"text": "Switch on the main light in the kitchen."}]
                    },
                    "final_response": {
                        "parts": [
                            {
                                "text": "Successfully set the main light in the kitchen to on."
                            }
                        ]
                    },
                    "intermediate_data": {
                        "tool_uses": [
                            {
                                "name": "set_device_status",
                                "args": {
                                    "location": "kitchen",
                                    "device_id": "main light",
                                    "status": "ON",
                                },
                            }
                        ]
                    },
                }
            ],
        },
    ],
}

In [11]:
import json

with open("home_automation_agent/integration.evalset.json", "w") as f:
    json.dump(test_cases, f, indent=2)

print("✅ Evaluation test cases created")
print("\n🧪 Test scenarios:")
for case in test_cases["eval_cases"]:
    user_msg = case["conversation"][0]["user_content"]["parts"][0]["text"]
    print(f"• {case['eval_id']}: {user_msg}")

print("\n📊 Expected results:")
print("• basic_device_control: Should pass both criteria")
print(
    "• wrong_tool_usage_test: May fail tool_trajectory if agent uses wrong parameters"
)
print(
    "• poor_response_quality_test: May fail response_match if response differs too much"
)

✅ Evaluation test cases created

🧪 Test scenarios:
• living_room_light_on: Please turn on the floor lamp in the living room
• kitchen_on_off_sequence: Switch on the main light in the kitchen.

📊 Expected results:
• basic_device_control: Should pass both criteria
• wrong_tool_usage_test: May fail tool_trajectory if agent uses wrong parameters
• poor_response_quality_test: May fail response_match if response differs too much


In [12]:
print("🚀 Run this command to execute evaluation:")
!adk eval home_automation_agent home_automation_agent/integration.evalset.json --config_file_path=home_automation_agent/test_config.json --print_detailed_results

🚀 Run this command to execute evaluation:
Using evaluation criteria: criteria={'tool_trajectory_avg_score': 1.0, 'response_match_score': 0.8} user_simulator_config=None
Tool Call: Setting floor lamp in living room to ON
Tool Call: Setting main light in kitchen to ON
*********************************************************************
Eval Run Summary
home_automation_integration_suite:
  Tests passed: 0
  Tests failed: 2
********************************************************************
Eval Set Id: home_automation_integration_suite
Eval Id: living_room_light_on
Overall Eval Status: FAILED
---------------------------------------------------------------------
Metric: tool_trajectory_avg_score, Status: PASSED, Score: 1.0, Threshold: 1.0
---------------------------------------------------------------------
Metric: response_match_score, Status: FAILED, Score: 0.0, Threshold: 0.8
---------------------------------------------------------------------
Invocation Details:
+----+--------------

C:\Python313\Lib\site-packages\google\adk\evaluation\metric_evaluator_registry.py:99: UserWarning: [EXPERIMENTAL] MetricEvaluatorRegistry: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  metric_evaluator_registry = MetricEvaluatorRegistry()
C:\Python313\Lib\site-packages\google\adk\evaluation\local_eval_service.py:83: UserWarning: [EXPERIMENTAL] UserSimulatorProvider: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  user_simulator_provider: UserSimulatorProvider = UserSimulatorProvider(),
C:\Python313\Lib\site-packages\google\adk\cli\cli_tools_click.py:762: UserWarning: [EXPERIMENTAL] UserSimulatorProvider: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  user_simulator_provider = UserSimulatorProvider(
C:\Pyth

In [13]:
# Analyzing evaluation results - the data science approach
print("📊 Understanding Evaluation Results:")
print()
print("🔍 EXAMPLE ANALYSIS:")
print()
print("Test Case: living_room_light_on")
print("  ❌ response_match_score: 0.45/0.80")
print("  ✅ tool_trajectory_avg_score: 1.0/1.0")
print()
print("📈 What this tells us:")
print("• TOOL USAGE: Perfect - Agent used correct tool with correct parameters")
print("• RESPONSE QUALITY: Poor - Response text too different from expected")
print("• ROOT CAUSE: Agent's communication style, not functionality")
print()
print("🎯 ACTIONABLE INSIGHTS:")
print("1. Technical capability works (tool usage perfect)")
print("2. Communication needs improvement (response quality failed)")
print("3. Fix: Update agent instructions for clearer language or constrained response.")
print()

📊 Understanding Evaluation Results:

🔍 EXAMPLE ANALYSIS:

Test Case: living_room_light_on
  ❌ response_match_score: 0.45/0.80
  ✅ tool_trajectory_avg_score: 1.0/1.0

📈 What this tells us:
• TOOL USAGE: Perfect - Agent used correct tool with correct parameters
• RESPONSE QUALITY: Poor - Response text too different from expected
• ROOT CAUSE: Agent's communication style, not functionality

🎯 ACTIONABLE INSIGHTS:
1. Technical capability works (tool usage perfect)
2. Communication needs improvement (response quality failed)
3. Fix: Update agent instructions for clearer language or constrained response.

